## Global alignment algorithms implementation
Alessandro Gennari

In [1]:
class typeException(Exception):
    def __init__(self, msg):
        print(msg)

## Needleman-Wunsch

In [11]:
def global_needleman_wunsch(match_cost, mismatch_cost, gap_cost, seq1, seq2):
    if isinstance(match_cost, int) == False or isinstance(match_cost, bool) == True:
        raise typeException(str(type(match_cost))+' is not the expected type for the variable for match_cost')
    if isinstance(mismatch_cost, int) == False or isinstance(mismatch_cost, bool) == True:
        raise typeException(str(type(mismatch_cost))+' is not the expected type for the variable for mismatch_cost')
    if isinstance(gap_cost, int) == False or isinstance(gap_cost, bool) == True:
        raise typeException(str(type(gap_cost))+' is not the expected type for the variable for gap_cost')
    if isinstance(seq1, str) == False:
        raise typeException(str(type(seq1))+' is not the expected type for the variable for seq1')
    if isinstance(seq2, str) == False:
        raise typeException(str(type(seq2))+' is not the expected type for the variable for seq2')
    if match_cost <= mismatch_cost:
        raise typeException('match_cost is <= than mismatch_cost!')

    import numpy as np

    scores = np.zeros((len(seq1)+1, len(seq2)+1))
    directions = scores.copy()

    # STEP 1 : Initialisation
    for i in range(len(seq1)+1):    # rows
        scores[i][0] = i * gap_cost
    for j in range(len(seq2)+1):    # columns
        scores[0][j] = j * gap_cost

    # STEP 2 : Matrixes Filling
    for i in range(1,len(seq1)+1):          # rows
        for j in range(1,len(seq2)+1):      # columns
            if seq1[i-1] == seq2[j-1]:
                diag = scores[i-1][j-1] + match_cost
            else:
                diag = scores[i-1][j-1] + mismatch_cost
            up = int(scores[i-1][j] + gap_cost)
            left = int(scores[i][j-1] + gap_cost)
            
            # diag, up, left are the score for each situation, now I choose the max and fill the directions matrix
            punteggi = np.array((diag, up, left))
            scores[i][j] = np.max(punteggi)
            directions[i][j] = np.argmax(punteggi)  # 0: diag, 1: up, 2: left

    # STEP 3: Trace back
    #print(f'{directions}\n{scores}')
    seq_ali_1 = ''
    seq_ali_2 = ''
    i = scores.shape[0]-1
    j = scores.shape[1]-1

    while (i > 0 and j > 0):
        if directions[i][j] == 0:       # diag
            seq_ali_1 = str(seq1[i-1]) + seq_ali_1
            seq_ali_2 = str(seq2[j-1]) + seq_ali_2
            j -= 1
            i -= 1
        elif directions[i][j] == 1:     # up
            seq_ali_1 = str(seq1[i-1]) + seq_ali_1
            seq_ali_2 = "-" + seq_ali_2 
            i -= 1
        elif directions[i][j] == 2:     # left
            seq_ali_1 = "-" + seq_ali_1
            seq_ali_2 = str(seq2[j-1]) + seq_ali_2
            j -= 1

    if (i == 0 and j != 0):       # meaning that seq2 starts with gaps
        while (j > 0):
            seq_ali_2 = "-" + seq_ali_2
            j -= 1
    if (j == 0 and i != 0):       # meaning that seq1 starts with gaps
        while (i > 0):
            seq_ali_1 = "-" + seq_ali_1
            i -= 1
    
    alignment = [seq_ali_1, seq_ali_2]
    return(alignment[0:2])

### TEST

In [29]:
seq1 = 'CCTATACCTAATTTTCGGCGCATGAGCCGGAATGGTGGGTACCGCTCTAAGCCTCCTCATTCGAGCAGAACTAGGCCAACCCGGAGCCCTTCTGGGAGACGACCAAGTCTACAACGTGGTTGTCACGGCCCATGCCTTCGTAATAATCTTCTTTATAGTTATGCCGATTATAATCGGAGGATTCGGAAACTGACTAGTCCCCCTAATAATCGGAGCCCCAGACATAGCATTTCCGCGAATAAACAACATAAGCTTCTGACTACTCCCACCATCATTCCTCCTCCTCTTAGCATCCTCCACAGTGGAAGCAGGCGTAGGTACAGGCTGAACAGTGTATCCCCCACTAGCTGGCAACCTAGCTCATGCCGGGGCCTCAGTCGACCTCGCAATCTTCTCCTTACACCTAGCTGGTATTTCCTCAATCCTCGGAGCAATTAACTTCATTACAACAGCAATTAACATGAAACCTCCTGCCCTCTCACAATACCAAACCCCACTATTCGTCTGATCAGTGTTAATTACTGCAGTCCTCCTTCTCCTTTCCCTTCCAGTTCTAGCTGCAGGAATCACAATGCTCCTCACAGACCGCAACCTCAACACCACATTCTTCGACCCTGCCGGAGGAGGAGATCCCGTCCTATATCAACATCTCTTCTGATTCTTCGGCCACCCAGAAGTCTACATCCTAATCCTC'
seq2 = 'GGAGACGACCAAATCTACAACGTAGTCGNCACGGCCCATGCTTCATGAGCTGGAATAGTAGGTACCGCCCTAAGCCTCCTAATTCGAGCAGAGCTAGGCCAACCCGGAGCCCTACTGTTGTAATAATCTTCTTCATAGTAATGCCAATCATAATCGGAGGGTTTGGAAACTGACTGGTCCCCCTAATAATTGGAGCTCCAGACATAGCATTCCCCCGAATAAACAACATGAGTTTCTGACTACTTCCCCCATCATTCCTACTACTAATAGCCTCCTCAACAGTAGAAGCAGGCGTTGGAACAGGATGAACCGTATATCCACCACTAGCCGGAAACCTAGCCCATGCAGGAGCCTCAGTAGACCTAGCTATCTTCTCCCTACACCTAGCAGGTATCTCCTCCATCCTAGGGGCAATCAACTTCATTACAACAGCAATCAACATAAAACCACCCGCCCTATCACAATACCAAACACCACTATTCGTATGATCCGTCCTAATCACAGCCGTACTACTCCTCCTATCACTCCCAGTGCTAGCTGCTGGAATTACCATGCTACTTACAGACCGCAACCTCAACACTACCTTCTTTGACCCAGCAGGGGGAGGAGACCCAGTGCTATACCAACATCTATTCTGATTCTTCGGACACCCAGAAGTTTACATCCTAATTCTC'
seq1 = 'ACGGTTGC'
seq2 = 'AGCGTC'
print(len(seq1))
print(len(seq2))

import time
start_time = time.time()
a = global_needleman_wunsch(1,-1,-2, seq1, seq2)
print("--- %s seconds ---" % (time.time() - start_time))
print(f'\n{a[0]}\n{a[1]}')


8
6
--- 0.0016832351684570312 seconds ---

ACGGTTGC
A-GCGT-C
